In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, ArrayType

# Define the schema for the price field
price_schema = StructType([
    StructField("raw", StringType(), True),
    StructField("formatted", StringType(), True),
    StructField("pricingOptionId", StringType(), True)
])

# Define the schema for origin and destination fields (they share the same structure)
location_schema = StructType([
    StructField("id", StringType(), True),
    StructField("entityId", StringType(), True),
    StructField("name", StringType(), True),
    StructField("displayCode", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("isHighlighted", BooleanType(), True)
])

# Define the schema for the carrier field
carrier_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("logoUrl", StringType(), True),
    StructField("name", StringType(), True)
])

# Define the schema for the segments field
segment_schema = StructType([
    StructField("id", StringType(), True),
    StructField("origin", location_schema, True),
    StructField("destination", location_schema, True),
    StructField("departure", StringType(), True),
    StructField("arrival", StringType(), True),
    StructField("durationInMinutes", IntegerType(), True),
    StructField("flightNumber", StringType(), True),
    StructField("marketingCarrier", carrier_schema, True),
    StructField("operatingCarrier", carrier_schema, True)
])

# Define the schema for the legs field
legs_schema = StructType([
    StructField("id", StringType(), True),
    StructField("origin", location_schema, True),
    StructField("destination", location_schema, True),
    StructField("durationInMinutes", IntegerType(), True),
    StructField("stopCount", IntegerType(), True),
    StructField("isSmallestStops", BooleanType(), True),
    StructField("departure", StringType(), True),
    StructField("arrival", StringType(), True),
    StructField("timeDeltaInDays", IntegerType(), True),
    StructField("carriers", StructType([
        StructField("marketing", ArrayType(carrier_schema), True),
        StructField("operationType", StringType(), True)
    ]), True),
    StructField("segments", ArrayType(segment_schema), True)
])

# Define the schema for the main JSON structure
schema = StructType([
    StructField("id", StringType(), True),
    StructField("price", price_schema, True),
    StructField("legs", ArrayType(legs_schema), True),
    StructField("isSelfTransfer", BooleanType(), True),
    StructField("isProtectedSelfTransfer", BooleanType(), True),
    StructField("farePolicy", StructType([
        StructField("isChangeAllowed", BooleanType(), True),
        StructField("isPartiallyChangeable", BooleanType(), True),
        StructField("isCancellationAllowed", BooleanType(), True),
        StructField("isPartiallyRefundable", BooleanType(), True)
    ]), True),
    StructField("fareAttributes", StructType([]), True),  # Assuming fareAttributes is empty
    StructField("tags", ArrayType(StringType()), True),
    StructField("isMashUp", BooleanType(), True),
    StructField("hasFlexibleOptions", BooleanType(), True),
    StructField("score", DoubleType(), True)
])


In [0]:
fromto.display()

id,entityId,flightPlaceType,localizedName,skyId,entityType,title,subtitle,id,entityId,flightPlaceType,localizedName,skyId,entityType,title,subtitle
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiSkZLIiwiZSI6Ijk1NTY1MDU4IiwiaCI6IjI3NTM3NTQyIn0=,95565058,AIRPORT,New York,JFK,CITY,New York John F. Kennedy,United States
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiU1dGIiwiZSI6Ijk1NTY2MjgwIiwiaCI6IjI3NTM3NTQyIn0=,95566280,AIRPORT,New York,SWF,CITY,Stewart International,United States
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiTEdBIiwiZSI6Ijk1NTY1MDU3IiwiaCI6IjI3NTM3NTQyIn0=,95565057,AIRPORT,New York,LGA,CITY,New York LaGuardia,United States
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiRVdSIiwiZSI6Ijk1NTY1MDU5IiwiaCI6IjI3NTM3NTQyIn0=,95565059,AIRPORT,New York,EWR,CITY,New York Newark,United States
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiTkNMIiwiZSI6Ijk1Njc0MDQ0IiwiaCI6IjI3NTQ1MDkyIn0=,95674044,AIRPORT,Newcastle,NCL,CITY,Newcastle,United Kingdom
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiTlFZIiwiZSI6Ijk1NjczOTYzIiwiaCI6IjI3NTQ1MTQ5In0=,95673963,AIRPORT,Newquay,NQY,CITY,Newquay,United Kingdom
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India,eyJzIjoiTllDQSIsImUiOiIyNzUzNzU0MiIsImgiOiIyNzUzNzU0MiJ9,27537542,CITY,New York,NYCA,CITY,New York,United States


In [0]:
from datetime import datetime
airport = spark.read.table('Airport')
# Get today's date in 'yyyy-MM-dd' format
today_date = datetime.today().strftime('%Y-%m-%d')
fromto = airport.alias("a1").crossJoin(airport.alias("a2")).filter("a1.id != a2.id")
# Create the 'flights' list
flights_data = fromto.selectExpr(
    "a1.id as fromId",
    "a2.id as toId",
    f'"{today_date}" as departDate'  # Using today's date
).rdd.map(lambda row: {
    "fromId": row.fromId,
    "toId": row.toId,
    "departDate": row.departDate
}).collect()



In [0]:
import requests
import json
from pyspark.sql.functions import *
def callAPI(indata):
    headers = {
    "x-rapidapi-host": "skyscanner80.p.rapidapi.com",
    "x-rapidapi-key": "da5e4cb143mshb0961ab3efe73aep1f7bb7jsnd5bbdf38c6dc"  
    }
    reqdata = {"market":"IN","locale":"en-GB","currency":"INR","adults":1,"children":0,"infants":0,"cabinClass":"economy","flights":[indata]}
    json_data=json.dumps(reqdata)
    response = requests.post('https://skyscanner80.p.rapidapi.com/api/v1/flights/search-multi-city', headers=headers,json=reqdata)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        flight=response.json()['data']['itineraries']
        rdd = spark.sparkContext.parallelize(flight)
        tempdf = spark.createDataFrame(rdd,schema=schema)
        return tempdf
    else:
        print(f"Error: {response.status_code}, {response.text}")


In [0]:
flightdf = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
for fd in flights_data:
    tempdf = callAPI(fd)
    flightdf=flightdf.union(tempdf)


In [0]:
flightdf.display()

id,price,legs,isSelfTransfer,isProtectedSelfTransfer,farePolicy,fareAttributes,tags,isMashUp,hasFlexibleOptions,score
12712-2409302030--31799-1-10957-2410012340,"List(40357, ₹40,357, SHs731JCnDQh)","List(List(12712-2409302030--31799-1-10957-2410012340, List(JFK, 95565058, New York John F. Kennedy, JFK, New York, United States, false), List(DEL, 95673498, Indira Gandhi International , DEL, New Delhi, India, false), 1060, 1, false, 2024-09-30T20:30:00, 2024-10-01T23:40:00, 1, List(List(List(-31799, https://logos.skyscnr.com/images/airlines/favicon/LX.png, SWISS)), fully_operated), List(List(12712-18563-2409302030-2410011025--31799, List(null, null, New York John F. Kennedy, JFK, null, United States, null), List(null, null, Zurich, ZRH, null, Switzerland, null), 2024-09-30T20:30:00, 2024-10-01T10:25:00, 475, 15, List(-31799, null, SWISS), List(-31799, null, SWISS)), List(18563-10957-2410011220-2410012340--31799, List(null, null, Zurich, ZRH, null, Switzerland, null), List(null, null, Indira Gandhi International , DEL, null, India, null), 2024-10-01T12:20:00, 2024-10-01T23:40:00, 470, 146, List(-31799, null, SWISS), List(-31799, null, SWISS)))))",false,false,"List(false, false, false, false)",List(),"List(second_cheapest, third_shortest)",false,false,0.999
12712-2409302300--32348-1-10957-2410020245,"List(46340, ₹46,340, N7G8WPvTn9ZT)","List(List(12712-2409302300--32348-1-10957-2410020245, List(JFK, 95565058, New York John F. Kennedy, JFK, New York, United States, false), List(DEL, 95673498, Indira Gandhi International , DEL, New Delhi, India, false), 1095, 1, false, 2024-09-30T23:00:00, 2024-10-02T02:45:00, 2, List(List(List(-32348, https://logos.skyscnr.com/images/airlines/favicon/EK.png, Emirates)), fully_operated), List(List(12712-11182-2409302300-2410011945--32348, List(null, null, New York John F. Kennedy, JFK, null, United States, null), List(null, null, Dubai, DXB, null, United Arab Emirates, null), 2024-09-30T23:00:00, 2024-10-01T19:45:00, 765, 202, List(-32348, null, Emirates), List(-32348, null, Emirates)), List(11182-10957-2410012200-2410020245--32348, List(null, null, Dubai, DXB, null, United Arab Emirates, null), List(null, null, Indira Gandhi International , DEL, null, India, null), 2024-10-01T22:00:00, 2024-10-02T02:45:00, 195, 512, List(-32348, null, Emirates), List(-32348, null, Emirates)))))",false,false,"List(false, false, false, false)",List(),null,false,false,0.868526
12712-2409302150--32090-1-10957-2410020055,"List(42106, ₹42,106, BO5p7t7mju9W)","List(List(12712-2409302150--32090-1-10957-2410020055, List(JFK, 95565058, New York John F. Kennedy, JFK, New York, United States, false), List(DEL, 95673498, Indira Gandhi International , DEL, New Delhi, India, false), 1055, 1, false, 2024-09-30T21:50:00, 2024-10-02T00:55:00, 2, List(List(List(-32090, https://logos.skyscnr.com/images/airlines/favicon/LH.png, Lufthansa)), fully_operated), List(List(12712-11616-2409302150-2410011145--32090, List(null, null, New York John F. Kennedy, JFK, null, United States, null), List(null, null, Frankfurt am Main, FRA, null, Germany, null), 2024-09-30T21:50:00, 2024-10-01T11:45:00, 475, 405, List(-32090, null, Lufthansa), List(-32090, null, Lufthansa)), List(11616-10957-2410011320-2410020055--32090, List(null, null, Frankfurt am Main, FRA, null, Germany, null), List(null, null, Indira Gandhi International , DEL, null, India, null), 2024-10-01T13:20:00, 2024-10-02T00:55:00, 485, 760, List(-32090, null, Lufthansa), List(-32090, null, Lufthansa)))))",false,false,"List(false, false, false, false)",List(),List(second_shortest),false,false,0.754818
12712-2409301730--32090-1-10957-2410012330,"List(40528, ₹40,528, 5QU9SVaTvv1C)","List(List(12712-2409301730--32090-1-10957-2410012330, List(JFK, 95565058, New York John F. Kennedy, JFK, New York, United States, false), List(DEL, 95673498, Indira Gandhi International , DEL, New Delhi, India, false), 1230, 1, false, 2024-09-30T17:30:00, 2024-10-01T23:30:00, 1, List(List(List(-32090, https://logos

In [0]:
from pyspark.sql.functions import col, explode, collect_set

# Explode legs and select relevant columns
df = flightdf.withColumn('flights', explode('legs')) \
    .drop('legs') \
    .select(
        col('flights.origin.displayCode').alias('SourceCode'),
        col('flights.origin.country').alias('SourceCountry'),
        col('flights.origin.name').alias('SourceAirport'),
        col('flights.destination.displayCode').alias('DestinationCode'),
        col('flights.destination.name').alias('DestinationAirport'),
        col('flights.durationInMinutes').alias('durationInMinutes'),
        col('flights.stopCount').alias('stopCount'),
        col('flights.departure').alias('departure'),
        col('flights.arrival').alias('arrival'),
        col('flights.timeDeltaInDays').alias('timeDeltaInDays'),
        col('Price.formatted').alias('Price'),
        col('farePolicy.isPartiallyRefundable').alias('isPartiallyRefundable'),
        col('flights.segments').alias('segments'),
        col('flights.carriers.marketing').alias('carriers'),
        'tags',
        'id'
    )

# Explode the segments and filter where SourceCode is not equal to Layover
df_layover = df.withColumn('segments', explode('segments')) \
    .withColumn('Layover', col('segments.origin.displayCode')) \
    .filter(col('SourceCode') != col('Layover')) \
    .groupBy('id').agg(collect_set('Layover').alias('Layovers'))

df_carrier = df.withColumn('carriers', explode('carriers'))\
             .withColumn('airline',col('carriers.name'))\
            .groupBy('id').agg(collect_set('airline').alias('airline'))
             


# Perform the join on 'id'
result_df = df.join(df_layover, on='id', how='outer').join(df_carrier, on='id', how='outer')\
            .drop('segments')\
            .drop('carriers')\
            .withColumn('DepartureDate', current_date())

# Display the result
result_df.display()


id,SourceCode,SourceCountry,SourceAirport,DestinationCode,DestinationAirport,durationInMinutes,stopCount,departure,arrival,timeDeltaInDays,Price,isPartiallyRefundable,tags,Layovers,airline,DepartureDate
10957-2409302330--31697-2-12712-2410012110,DEL,India,Indira Gandhi International,JFK,New York John F. Kennedy,1870,2,2024-09-30T23:30:00,2024-10-01T21:10:00,1,"₹75,789",false,"List(third_cheapest, third_shortest)","List(BOM, LHR)",List(Virgin Atlantic),2024-09-30
10957-2409302330--31697-2-12712-2410012110,DEL,India,Indira Gandhi International,JFK,New York John F. Kennedy,1870,2,2024-09-30T23:30:00,2024-10-01T21:10:00,1,"₹75,789",false,null,"List(BOM, LHR)",List(Virgin Atlantic),2024-09-30
10957-2409302330--32213-2-12712-2410011730,DEL,India,Indira Gandhi International,JFK,New York John F. Kennedy,1650,2,2024-09-30T23:30:00,2024-10-01T17:30:00,1,"₹49,091",false,"List(cheapest, second_shortest)","List(BOM, IST)",List(IndiGo),2024-09-30
10957-2409302330--32213-2-12712-2410011730,DEL,India,Indira Gandhi International,JFK,New York John F. Kennedy,1650,2,2024-09-30T23:30:00,2024-10-01T17:30:00,1,"₹71,448",false,"List(third_cheapest, third_shortest)","List(BOM, IST)",List(IndiGo),2024-09-30
10957-2409302330--32573-0-12712-2410010620,DEL,India,Indira Gandhi International,JFK,New York John F. Kennedy,980,0,2024-09-30T23:30:00,2024-10-01T06:20:00,1,"₹54,831",false,"List(second_cheapest, shortest)",null,List(American Airlines),2024-09-30
10957-2409302330--32573-0-12712-2410010620,DEL,India,Indira Gandhi International,JFK,New York John F. Kennedy,980,0,2024-09-30T23:30:00,2024-10-01T06:20:00,1,"₹54,774",false,"List(cheapest, shortest)",null,List(American Airlines),2024-09-30
10957-2409302335--31722-0-11442-2410010625,DEL,India,Indira Gandhi International,EWR,New York Newark,980,0,2024-09-30T23:35:00,2024-10-01T06:25:00,1,"₹68,227",false,"List(second_cheapest, second_shortest)",null,List(United),2024-09-30
10957-2409302335--31722-0-11442-2410010625,DEL,India,Indira Gandhi International,EWR,New York Newark,980,0,2024-09-30T23:35:00,2024-10-01T06:25:00,1,"₹68,227",false,"List(second_cheapest, second_shortest)",null,List(United),2024-09-30
10957-2409302335--32090-0-11442-2410010625,DEL,India,Indira Gandhi International,EWR,New York Newark,980,0,2024-09-30T23:35:00,2024-10-01T06:25:00,1,"₹66,697",false,"List(cheapest, shortest)",null,List(Lufthansa),2024-09-30
11442-2409301700--31697-1-14585-2410011035,EWR,United States,New York Newark,NCL,Newcastle,755,1,2024-09-30T17:00:00,2024-10-01T10:35:00,1,"₹87,333",false,null,List(CDG),List(Virgin Atlantic),2024-09-30


In [0]:
result_df.write.mode('append').saveAsTable('Flights')